In [ ]:
!pip install transformers
!pip install accelerate
!pip install datasets
!pip install bitsandbytes
!pip install peft
!pip install trl
!pip install optimum
!pip install auto-gptq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.3/168.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 403.3/403.3 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

### Load Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("HuggingFaceH4/ultrafeedback_binarized",
                        split=["train_prefs", "test_prefs"],
                        use_auth_token=True)
train_dataset = dataset[0]
valid_dataset = dataset[1]

/usr/local/lib/python3.10/dist-packages/datasets/load.py:2479: FutureWarning: 'use_auth_token' was deprecated in favor of 'token' in version 2.14.0 and will be removed in 3.0.0.
You can remove this warning by passing 'token=<use_auth_token>' instead.
  warnings.warn(


Generating train_sft split:   0%|          | 0/61966 [00:00<?, ? examples/s]

Generating test_sft split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_gen split:   0%|          | 0/61966 [00:00<?, ? examples/s]

Generating test_gen split:   0%|          | 0/1000 [00:00<?, ? examples/s]

Generating train_prefs split:   0%|          | 0/61966 [00:00<?, ? examples/s]

Generating test_prefs split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
train_dataset["prompt"][0]

'Write a 1,000-word op-ed piece in a formal tone, analyzing and providing examples of the ways in which social media platforms have been utilized to spread extremist and violent ideologies. In your analysis, discuss the specific tactics that these groups use to spread their messages online and the effects of these tactics on both individuals and society. Additionally, provide possible solutions that could be implemented to combat the spread of these dangerous ideologies on social media. Your piece should be well-researched, citing reputable sources to support your arguments.'

In [ ]:
train_dataset["chosen"][0][1]

{'content': "Title: The Noxious Alliance of Social Media and Extremism: A Threat to Human Progress\n\nThe advent of social media platforms has irrevocably revolutionized the way we communicate, share and consume information. While these platforms provide access to information for a global audience and allow individuals to connect regardless of geographical barriers, they have also become a breeding ground for the spread of extremist and violent ideologies, presenting a significant threat to human progress. The pervasiveness of social media, coupled with the anonymity afforded by these platforms, has made it easier for extremists to disseminate their messages, recruit new members, and incite violence against targeted groups, resulting in devastating consequences.\n\nExtremist groups often exploit the viral nature of social media to propagate their ideologies. Fast-paced videos, catchy slogans, and emotionally-charged content are a few of the strategies employed by these groups. Swift an

In [ ]:
train_dataset["rejected"][0][1]

{'content': '**The Spread of Extremist and Violent Ideologies on Social Media**\n\nSocial media platforms have become a powerful tool for spreading extremist and violent ideologies. These platforms provide a way for extremists to connect with like-minded individuals, to spread their messages to a wider audience, and to recruit new members.\n\nExtremists use social media in a variety of ways to spread their messages. One common tactic is to create and share content that is designed to appeal to young people. This content often takes the form of memes, videos, and images that are designed to be funny, shocking, or provocative. Extremists also use social media to create echo chambers where they can share their messages without being challenged or criticized. This can make it difficult for people to escape the pull of extremist ideologies.\n\nThe spread of extremist and violent ideologies on social media can have a number of negative effects. These ideologies can lead to violence, both onl

In [ ]:
def dpo_data(train_dataset, valid_dataset):
    original_columns = train_dataset.column_names

    def return_prompt_and_responses(samples):
        return {
            "prompt": [prompt for prompt in samples["prompt"]],
            "chosen": samples["chosen"],
            "rejected": samples["rejected"]
        }
    train_dataset = train_dataset.map(return_prompt_and_responses,
                                      batched=True,
                                      remove_columns=original_columns)

    valid_dataset = valid_dataset.map(return_prompt_and_responses,
                                      batched=True,
                                      remove_columns=original_columns)

    return train_dataset, valid_dataset


In [ ]:
train_dataset, valid_dataset = dpo_data(train_dataset, valid_dataset)

Map:   0%|          | 0/61966 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
train_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 61966
})

In [ ]:
train_df = train_dataset.to_pandas()
valid_df = valid_dataset.to_pandas()

train_df

,prompt,chosen,rejected
0,"Write a 1,000-word op-ed piece in a formal ton...","[{'content': 'Write a 1,000-word op-ed piece i...","[{'content': 'Write a 1,000-word op-ed piece i..."
1,Using a range of physical activities and creat...,[{'content': 'Using a range of physical activi...,[{'content': 'Using a range of physical activi...
2,Which books are still banned in Canada?,[{'content': 'Which books are still banned in ...,[{'content': 'Which books are still banned in ...
3,People watch a soccer player in uniform make a...,[{'content': 'People watch a soccer player in ...,[{'content': 'People watch a soccer player in ...
4,What is the advantage of separable filters?,[{'content': 'What is the advantage of separab...,[{'content': 'What is the advantage of separab...
...,...,...,...
61961,Write an article about the html buttonPlease i...,[{'content': 'Write an article about the html ...,[{'content': 'Write an article about the html ...
61962,Explain what is an algorithmic trading.,[{'content': 'Explain what is an algorithmic t...,[{'content': 'Explain what is an algorithmic t...
61963,Determine the median of the following group of...,[{'content': 'Determine the median of the foll...,[{'content': 'Determine the median of the foll...
61964,You are given a sentence in Arabic. Your job i...,[{'content': 'You are given a sentence in Arab...,[{'content': 'You are given a sentence in Arab...


In [ ]:
def select_df_content(df):
    df["chosen"] = df["chosen"].apply(lambda x: x[1]["content"])
    df["rejected"] = df["rejected"].apply(lambda x: x[1]["content"])

    df.dropna(inplace=True)

    return df

In [ ]:
train_df = select_df_content(train_df)
valid_df = select_df_content(valid_df)

In [ ]:
from datasets import Dataset

train_data = Dataset.from_pandas(train_df)
valid_data = Dataset.from_pandas(valid_df)

In [ ]:
train_data

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 61966
})

In [ ]:
model_ckpt_path = "TheBloke/OpenHermes-2-Mistral-7B-GPTQ"

### Tokenizer

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_ckpt_path)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/1.42k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/174 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


### GPTQ Config

In [ ]:
from transformers import GPTQConfig

quantization_config = GPTQConfig(bits=4,
                                 use_exllama=False)

### Causal LM Model

In [ ]:
import torch
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(model_ckpt_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=quantization_config)
model_ref = AutoModelForCausalLM.from_pretrained(model_ckpt_path, torch_dtype=torch.float16, low_cpu_mem_usage=True, quantization_config=quantization_config)

config.json:   0%|          | 0.00/1.32k [00:00<?, ?B/s]

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


model.safetensors:   0%|          | 0.00/4.16G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute and has already quantized weights. However, loading attributes (e.g. use_exllama, exllama_config, use_cuda_fp16, max_input_length) will be overwritten with the one you passed to `from_pretrained`. The rest will be ignored.


In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096, padding_idx=0)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (rotary_emb): MistralRotaryEmbedding()
          (k_proj): QuantLinear()
          (o_proj): QuantLinear()
          (q_proj): QuantLinear()
          (v_proj): QuantLinear()
        )
        (mlp): MistralMLP(
          (act_fn): SiLUActivation()
          (down_proj): QuantLinear()
          (gate_proj): QuantLinear()
          (up_proj): QuantLinear()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=32002, bias=False)
)

### Lora Config

In [ ]:
from peft import LoraConfig

peft_config = LoraConfig(r=8,
                         lora_alpha=8,
                         lora_dropout=0.1,
                         target_modules=["q_proj", "v_proj"],
                         bias="none",
                         task_type="CAUSAL_LM")
peft_config.inference_mode = False

In [ ]:
from peft import get_peft_model, prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)
model.config.use_cache = False
model.gradient_checkpointing_enable()
model.config.pretraining_tp = 1
model = get_peft_model(model, peft_config)

In [ ]:
model.print_trainable_parameters()

trainable params: 3,407,872 || all params: 265,834,496 || trainable%: 1.2819525122879463


### Training Arguments

In [ ]:
from transformers import TrainingArguments

output_dir="openhermes-2-mistral-dpo-gptq"

training_args = TrainingArguments(output_dir=output_dir,
                                  per_device_train_batch_size = 1,
                                  max_steps=50,
                                  remove_unused_columns=False,
                                  gradient_accumulation_steps=1,
                                  learning_rate=2e-4,
                                  evaluation_strategy="steps",
                                  num_train_epochs=10,
                                  logging_first_step=True,
                                  logging_steps=10,
                                  optim="paged_adamw_32bit",
                                  warmup_steps=2,
                                  fp16=True,
                                  push_to_hub=True)

### DPOTrainer

In [ ]:
from trl import DPOTrainer

dpo_trainer = DPOTrainer(model,
                         model_ref,
                         args=training_args,
                         beta=0.1,
                         train_dataset=train_data,
                         eval_dataset=valid_data,
                         tokenizer=tokenizer,
                         max_length=512,
                         max_target_length=256,
                         max_prompt_length=256)

In [ ]:
dpo_trainer.train()

In [ ]:
dpo_trainer.push_to_hub()

### Inference

In [ ]:
import torch
from transformers import AutoTokenizer
from transformers import GenerationConfig
from peft import AutoPeftModelForCausalLM

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("MohamedAhmedAE/mistral-dpo-gptq")
inputs = tokenizer("""I have dropped my phone in water. Now it is not working what should I do now?""", return_tensors="pt").to("cuda")

model = AutoPeftModelForCausalLM("MohamedAhmedAE/mistral-dpo-gptq",
                                 low_cpu_mem_usage=True,
                                 return_dict=True,
                                 torch_dtype=torch.float16,
                                 device_map="cuda")

generation_config = GenerationConfig(do_sample=True,
                                     top_k=1,
                                     temperature=0.1,
                                     max_new_tokens=256,
                                     pad_token_id=tokenizer.eos_token_id)

In [ ]:
from time import perf_counter

start_time = perf_counter()
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
print(perf_counter()-start_time)